In [5]:
import re,argparse
import sys
from matplotlib import pyplot
import plistlib
import numpy as np
import pprint
from collections import Counter
def findDuplicates(fileName):
    print('Finding duplicate tracks in %s...'%fileName)
    #read in a playlist
    plist=plistlib.readPlist(fileName)
    '''
    {'Major Version': 1, 'Minor Version': 1.......
    'Tracks': {'141': {'Track ID': 141, 'Name': 'Angelina',.......    
               '146': {'Track ID': 146, 'Name': '莫斯科没有眼泪',
    '''
    #get the tracks from the Tracks dictionary
    tracks=plist['Tracks']
    #create a track name dictionary
    trackNames={}
    #叠代tracks
    for trackId,track in tracks.items():
        '''what in tracks?
        'Tracks': {'141': {'Album': 'Ladies And Gentlemen',
                    'Artist': 'Lou Bega',
                    'Bit Rate': 128,
                    'Date Added': datetime.datetime(2020, 9, 29, 2, 8, 14),
                    'Date Modified': datetime.datetime(2019, 12, 4, 9, 28, 7),
                    'File Folder Count': -1,
                    'Genre': '20953761_1941006',
                    'Kind': 'MPEG 音频文件',
                    'Library Folder Count': -1,
                    'Location': 'file://localhost/C:/Users/xxcmt/Music/%E5%AE%89%E5%90%89%E4%B8%BD%E5%A8%9C.mp3',
                    'Name': 'Angelina',
                    'Persistent ID': 'D057CA97CCCCDB90',
                    'Sample Rate': 44100,
                    'Size': 3154934,
                    'Total Time': 197172,
                    'Track ID': 141,
                    'Track Type': 'File'},
        '''
        try:
            name=track['Name']
            duration=track['Total Time']
            #look for existing entries
            if name in trackNames:
                #if a name and duration match,increment the count
                #round the track length to the nearest second
                '''the default trackNames is {name:(duaration,1)}'''
                if durattion==trackNames[name][0]:
                    count=trackNames[name][1]
                    trackNames[name]=(duration,count+1)
            else:
                #add dictionary entry as tuple(duration,count)
                trackNames[name]=(duration,1)
                '''so trackNames is {name:(duaration,1)}
                '''
        except:
            #ignore
            pass
    dups=[]
    for k,v in trackNames.items():
        #the trackNames is {name:(duaration,1)}
        #the v[1] is the default 1
        if v[1]>1:
            dups.append((v[1],k))
            #the dups is [('1+',name),...]
    if len(dups)>0:
        print("Found %d duplicates.Track names saved to dup.txt"%len(dups))
        with open("dups.txt","w") as f:
            for val in dups:
                f.write("[%d]%s\n"%(val[0],val[1]))
    else:
        print("No duplicate track found!")

#to find commontracks in diffrent filenames like to find common intrest in diffrent people        
def findCommonTracks(fileNames):
    #a list of sets of track names
    trackNameSets=[]
    #给出一个空字典
    trackNamesDict={}
    trackNamesCommon=[]
    for fileName in fileNames:
        #read in playlist
        plist=plistlib.readPlist(fileName)
        #get the tracks
        tracks=plist['Tracks']
        '''
                   what's in tracks
        'Tracks': {'141': 
                   {'Album': 'Ladies And Gentlemen',
                    'Artist': 'Lou Bega',
                    'Bit Rate': 128,
                    'Date Added': datetime.datetime(2020, 9, 29, 2, 8, 14),
                    'Date Modified': datetime.datetime(2019, 12, 4, 9, 28, 7),
                    'File Folder Count': -1,
                    'Genre': '20953761_1941006',
                    'Kind': 'MPEG 音频文件',
                    'Library Folder Count': -1,
                    'Location': 'file://localhost/C:/Users/xxcmt/Music/%E5%AE%89%E5%90%89%E4%B8%BD%E5%A8%9C.mp3',
                    'Name': 'Angelina',
                    'Persistent ID': 'D057CA97CCCCDB90',
                    'Sample Rate': 44100,
                    'Size': 3154934,
                    'Total Time': 197172,
                    'Track ID': 141,
                    'Track Type': 'File'},
        '''
        #iterate through the tracks
        for trackId,track in tracks.items():
            '''trackId是141，
                 track是{
                 'Album': 'Ladies And Gentlemen',
                'Artist': 'Lou Bega'
                  'Name': 'Angelina',}
            '''
            try:
                #track['Name']：(duration,1)
                name=track['Name']
                duration=track['Total Time']
                #print(duration)
                if name in trackNamesDict:
                #if a name and duration match,increment the count
                #round the track length to the nearest second
                #the default trackNames is {name:(duaration,count)}
                    if duration in trackNamesDict[name]:
                        trackNamesCommon.append(name)
                    else:
                        trackNamesDict[name].append(duration)
                else:
                    #add dictionary entry as tuple(duration,count)
                    trackNamesDict[name]=[duration]
                    '''so trackNames is {name:(duaration,1)}
                    '''
            except:
                pass
    if len(trackNamesCommon)>0:
            with open(r"C:\\Users\\xxcmt\\Desktop\\暂存\\common.txt","w") as f:
                for v in trackNamesCommon:
                    s="%s\n"%v
                    f.write(s)
            print("%d common tracks found."
              "Track names written to common.txt."%len(trackNamesCommon))
    else:
        print("No common tracks!")
    pprint.pprint(trackNamesCommon)
    namesDiffDurations=[k for k,v in trackNamesDict.items() if len(v)>1]
    if len(namesDiffDurations)>0:
            with open(r"C:\\Users\\xxcmt\\Desktop\\暂存\\namesDiffDurations.txt","w") as f:
                for v in namesDiffDurations:
                    s="%s\n"%v
                    f.write(s)
            print("%d same name but Diff Durations tracks found."
              "Track names written to namesDiffDurations.txt."%len(namesDiffDurations))
    else:
        print("No same name but Diff Durations tracks!")
    pprint.pprint(namesDiffDurations)

    
        
    #add to list
'''trackNameSets.append(trackNames)
    #get the set of common tracks
    #*trackNameSets作用是将列表解开成独立的参数，传入函数
    commonTracks=set.intersection(*trackNameSets)
    #write to file
    commontracksnames=[]
    #pprint.pprint(trackNamesDict)
    #pprint.pprint(commonTracks)
    if len(commonTracks)>0:
        for val in commonTracks:
            if trackNamesDict[val][1]>1:
                commontracksnames.append(val)
        if len(commontracksnames)>0:
            with open("common.txt","w") as f:
                for v in commontracksnames:
                    s="%s\n"%v
                    #f.write(s.encode("UTF-8"))
                    f.write(s)
            print("%d common tracks found."
              "Track names written to common.txt."%len(commonTracks))
        else:
            print ("No common tracks!")
    else:
        print("No common tracks!")
'''
        
def plotStats(fileName):
    #read in a playlist
    plist=plistlib.readPlist(fileName)
    #get the tracks from the playlist
    tracks=plist['Tracks']
    '''
        'Tracks': 
        {'141': 
                【what's in tracks】
                {'Album': 'Ladies And Gentlemen',
                'Artist': 'Lou Bega',
                'Bit Rate': 128,
                'Date Added': datetime.datetime(2020, 9, 29, 2, 8, 14),
                'Date Modified': datetime.datetime(2019, 12, 4, 9, 28, 7),
                'File Folder Count': -1,
                'Genre': '20953761_1941006',
                'Kind': 'MPEG 音频文件',
                'Library Folder Count': -1,
                'Location': 'file://localhost/C:/Users/xxcmt/Music/%E5%AE%89%E5%90%89%E4%B8%BD%E5%A8%9C.mp3',
                'Name': 'Angelina',
                'Persistent ID': 'D057CA97CCCCDB90',
                'Sample Rate': 44100,
                'Size': 3154934,
                'Total Time': 197172,
                'Track ID': 141,
                'Track Type': 'File'},
    '''
    #create lists of song ratings and track durations
    ratings=[]
    durations=[]
    #iterate through the tracks
    for trackId,track in tracks.items():
        try:
            ratings.append(track['Album Rating'])
            durations.append(track['Total Time'])
        except:
            #ignore
            pass
            
    #ensure that valid data was collected
    if ratings==[] or durations==[]:
        print('No valid Album Rating/Total Time data in %s.'%fileName)
        return
    #scatter plot,x轴的点由总时间组成
    x=np.array(durations,np.int32)
    #convert to minutes
    x=x/60000
    #y轴的点由唱片评分组成
    y=np.array(ratings,np.int32)
    #拆成两个图，上图如是显示
    pyplot.subplot(2,1,1)
    #o表示用圆实心点来表示
    pyplot.plot(x,y,'o')
    #设置轴的区间x为0至1.05倍的最大x;y为-1至110
    pyplot.axis([0,1.05*np.max(x),-1,110])
    #设置x的标签为。。。
    pyplot.xlabel('Track duration')
    #设置y轴的标签为
    pyplot.ylabel('Track rating')
    
    #plot histogram，设置第二个图
    pyplot.subplot(2,1,2)
    #pyplot.axis([0,1.05*np.max(x),0,x.itemsize)使用相同的x轴就好
    #画直方图，等分为20个直方，bins表示分为多少个直方
    pyplot.hist(x,bins=2000)
    pyplot.xlabel('Track duration')
    pyplot.ylabel('Count')
    #show plot
    pyplot.show()
    
            
    
def main():
    #creat parser
    descStr="this program analyzes playlist files (.xml) exported form iTunes."
    parser=argparse.ArgumentParser(description=descStr)
    #add a mutually exclusive group of arguments
    group=parser.add_mutually_exclusive_group()
    
    #add expected arguments,required为False表示该选项可以省略，否则不能省略
    group.add_argument('--common',nargs='*',dest='plFiles',required=False)
    #dest表示一个替代名称，required为true时为必选参数，metavar变更别名，dest自定义属性名
    group.add_argument('--stats',dest='plFile',required=False)
    group.add_argument('--dup',dest='plFileD',required=False)
    #parse args
    args=parser.parse_args()
    if args.plFiles:
        #find common tracks
        findCommonTracks(args.plFiles)
    elif args.plFile:
        #polt stats
        plotStats(args.plFile)
    elif args.plFileD:
        findDuplicates(args.plFileD)
    else:
        print('These are not the tracks you are looking for.')
    
if __name__=='__main__':
    main()
    
    
    

Finding duplicate tracks in C:\Users\xxcmt\Music\资料库.xml...
{'Application Version': '12.10.9.3',
 'Date': datetime.datetime(2020, 9, 29, 12, 55, 25),
 'Features': 5,
 'Library Persistent ID': '6D24F6682D3A4498',
 'Major Version': 1,
 'Minor Version': 1,
 'Music Folder': 'file://localhost/C:/Users/xxcmt/Music/iTunes/iTunes%20Media/',
 'Playlists': [{'All Items': True,
                'Description': '',
                'Master': True,
                'Name': '资料库',
                'Playlist ID': 68,
                'Playlist Items': [{'Track ID': 141},
                                   {'Track ID': 146},
                                   {'Track ID': 149}],
                'Playlist Persistent ID': '5040D9EDD6BE155E',
                'Visible': False},
               {'All Items': True,
                'Description': '',
                'Distinguished Kind': 65,
                'Name': '已下载',
                'Playlist ID': 105,
                'Playlist Items': [{'Track ID': 141},
    

C:\Users\xxcmt\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: The readPlist function is deprecated, use load() instead
  # This is added back by InteractiveShellApp.init_path()


In [17]:
p=(1,2)
p[0]=p[0]+1
print(p)

TypeError: 'tuple' object does not support item assignment

In [2]:

x = {"a", "b", "c","c", "d","e","f", "g", "d"}
y = {"c", "d", "e"}
z = {"f", "g", "d"}
result=[]
for i in (x,y,z):
    result.append(i)
print(result)
common = set.intersection(*result) 
print(common)
print(type(x))


[{'b', 'e', 'a', 'd', 'f', 'c', 'g'}, {'e', 'd', 'c'}, {'d', 'f', 'g'}]
{'d'}
<class 'set'>
